In [7]:
import pickle
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

# Import and Process Classification Results

## Intra-subject

In [5]:
file_names = os.listdir('./outputs/classification/intra-subject')
group_dict = {}
for file_name in file_names:
    key  = file_name.split('_Subject', 1)[0]
    if key not in group_dict.keys():
        group_dict[key] = [file_name]
    else:
        group_dict[key].append(file_name)

### Mean Performance

In [8]:
mean_results_dict = {}
for key in group_dict.keys():
    report_list = []
    for file in group_dict[key]:
        with open('./outputs/classification/intra-subject/'+file, 'rb') as infile:
            class_dict = pickle.load(infile)
            sample_result = class_dict['TruePredY']
            sample_reports = classification_report(sample_result['True'], sample_result['Predicted'], output_dict=True)
            sample_reports_df = pd.DataFrame(sample_reports).drop(columns=['macro avg', 'weighted avg'], index='support')
            report_list.append(sample_reports_df)
    mean_results_dict[key] = pd.concat(report_list).reset_index().groupby('index').mean().round(decimals=2).reset_index()
pd.concat(list(mean_results_dict.values())).to_csv('./report_intraSubject.csv')

In [9]:
pd.concat(list(mean_results_dict.values()))

,index,electricPanel,hoist,ladder,lift,overhead,push,sit,stand,type,walk,accuracy
0,f1-score,0.86,0.78,0.78,0.76,0.83,0.88,0.97,0.92,0.95,0.98,0.86
1,precision,0.84,0.92,0.76,0.71,0.94,0.88,0.98,0.93,0.96,1.00,0.86
2,recall,0.91,0.69,0.80,0.84,0.76,0.89,0.98,0.93,0.94,0.96,0.86
0,f1-score,0.90,0.84,0.85,0.81,0.74,0.93,0.99,0.98,0.95,1.00,0.90
1,precision,0.90,0.99,0.88,0.78,0.93,0.87,0.98,0.99,0.93,1.00,0.90
2,recall,0.92,0.76,0.84,0.88,0.66,1.00,0.99,0.98,0.99,1.00,0.90
0,f1-score,0.92,0.88,0.87,0.86,0.89,0.95,0.98,0.95,0.95,0.97,0.92
1,precision,0.91,0.95,0.86,0.84,0.95,0.94,0.98,0.97,0.96,0.96,0.92
2,recall,0.94,0.83,0.89,0.90,0.84,0.96,0.97,0.94,0.93,0.98,0.92
0,f1-score,0.90,0.94,0.95,0.94,0.69,0.92,0.99,0.98,0.98,0.99,0.94


## Inter-subject

In [23]:
resultsDict = {}
for file in os.listdir('./outputs/classifictaion/inter-subject'):
    infile = open('./outputs/classifictaion/inter-subject/' + file, 'rb')
    resultsDict['{}'.format(file)] = pickle.load(infile)
    infile.close()

In [25]:
for key in resultsDict.keys():
    Y_true_pred = resultsDict[key]['TruePredY']
    best_accuracy = accuracy_score(Y_true_pred.values[:,0], Y_true_pred.values[:,1])
    print('Test accuracy for {} is: {}.'.format(key, best_accuracy))

Test accuracy for FrequencyAll_KNN_128.pickle is: 0.828149474155292.
Test accuracy for FrequencyAll_KNN_320.pickle is: 0.8985915492957747.
Test accuracy for FrequencyAll_RandomForest_128.pickle is: 0.8543298277019468.
Test accuracy for FrequencyAll_RandomForest_320.pickle is: 0.9256338028169014.
Test accuracy for FrequencyAll_SVC_128.pickle is: 0.8784963078988588.
Test accuracy for FrequencyAll_SVC_320.pickle is: 0.948169014084507.
Test accuracy for TimeFrequency_KNN_128.pickle is: 0.9093756992615798.
Test accuracy for TimeFrequency_KNN_320.pickle is: 0.9622535211267605.
Test accuracy for TimeFrequency_RandomForest_128.pickle is: 0.8916983665249496.
Test accuracy for TimeFrequency_RandomForest_320.pickle is: 0.9656338028169014.
Test accuracy for TimeFrequency_SVC_128.pickle is: 0.9178787200716044.
Test accuracy for TimeFrequency_SVC_320.pickle is: 0.967887323943662.
Test accuracy for Time_KNN_128.pickle is: 0.8498545535914075.
Test accuracy for Time_KNN_320.pickle is: 0.905352112676056

### Table of Metrics for a Sample Model

In [14]:
sample_result = resultsDict['FrequencyAll_KNN_128']['TruePredY']
sample_reports = classification_report(sample_result['True'], sample_result['Predicted'], digits=3, output_dict=True)
sample_reports_df = pd.DataFrame(sample_reports)
sample_reports_df = sample_reports_df.round(decimals=2)
sample_reports_df.loc['support'] = sample_reports_df.loc['support'].astype(int).astype(str)
sample_reports_df = sample_reports_df.drop(columns=['accuracy', 'macro avg', 'weighted avg'])
sample_reports_df

,electricPanel,hoist,ladder,lift,overhead,push,sit,stand,type,walk
precision,0.83,0.96,0.7,0.76,0.91,0.85,0.81,0.89,0.86,1.0
recall,0.87,0.69,0.82,0.84,0.69,0.87,0.94,0.87,0.76,0.96
f1-score,0.85,0.8,0.76,0.8,0.78,0.86,0.87,0.88,0.81,0.98
support,360,441,862,516,360,493,360,360,360,357


### Create Table of Metrics for all Models

In [26]:
allReports_df = pd.DataFrame(columns=['electricPanel', 'hoist', 'ladder', 'lift', 'overhead', 'push', 'sit', 'stand', 'type', 'walk'])
for key in resultsDict.keys():
    truePredY = resultsDict[key]['TruePredY']
    reports = classification_report(truePredY['True'], truePredY['Predicted'], output_dict=True)
    reports_df = pd.DataFrame(reports)
    reports_df = reports_df.round(decimals=2)
    reports_df.loc['support'] = reports_df.loc['support'].astype(int).astype(str)
    reports_df = reports_df.drop(columns=['accuracy', 'macro avg', 'weighted avg'])
    allReports_df = pd.concat([allReports_df, reports_df], axis=0)
allReports_df.to_csv('./report_interSubject.csv')